In [1]:
import pandas as pd #working with DataFrames
import seaborn as sns #
import matplotlib.pyplot as plt
import copy
import time
sns.set_theme()

df_r = pd.read_csv('../data/cln_rotten_tomatoes_critic_reviews.csv')
df_m = pd.read_csv('../data/cln_rotten_tomatoes_movies.csv')

df_m.loc[0,'genres']

"['Action & Adventure', 'Comedy', 'Drama', 'Science Fiction & Fantasy']"

In [2]:
def destring(column):
    df_m[column] = df_m[column].map(lambda x: [y.strip("'") for y in x.strip('][').split(', ')])

In [3]:
df_m.head(2)

,Unnamed: 0,idx,rotten_tomatoes_link,movie_title,content_rating,genres,directors,authors,actors,runtime,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,0,0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,PG,"['Action & Adventure', 'Comedy', 'Drama', 'Sci...",['Chris Columbus'],"['Craig Titley', 'Chris Columbus', 'Rick Riord...","['Logan Lerman', 'Brandon T. Jackson', 'Alexan...",119.0,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,1,1,m/0878835,Please Give,R,['Comedy'],['Nicole Holofcener'],['Nicole Holofcener'],"['Catherine Keener', 'Amanda Peet', 'Oliver Pl...",90.0,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19


In [4]:
for column in ['genres','directors','authors','actors']:
    destring(column)

print(type(df_m.loc[0,'genres']), type(df_m.loc[0,'actors']))

<class 'list'> <class 'list'>


In [5]:
df_m.drop(['Unnamed: 0','idx'], axis = 1, inplace = True)
df_m = df_m.loc[df_m['runtime'] >= 45]

In [6]:
df_m_bins = copy.deepcopy(df_m)
df_m_bins['audience_count'] = (df_m_bins['audience_count']//100)*100
print(f"Mean: {df_m_bins['audience_count'].mean()}, Median: {df_m_bins['audience_count'].median()}, Mode: {df_m_bins['audience_count'].mode()}")

Mean: 147451.29927523423, Median: 4400.0, Mode: 0    0.0
dtype: float64


In [7]:
print(df_m.shape)
df_m = df_m.loc[df_m['audience_count'] >= df_m['audience_count'].quantile(0.75)]
df_m.shape

(16971, 18)


(4243, 18)

In [8]:
df_r.head()

,version https://git-lfs.github.com/spec/v1
0,oid sha256:49da1ba1d4ecccf4155c623fb8e802350bc...
1,size 106185707


In [9]:
df_r.drop(['Unnamed: 0','idx'], axis = 1, inplace = True)

KeyError: "['Unnamed: 0' 'idx'] not found in axis"

In [ ]:
df = df_m.set_index('rotten_tomatoes_link').join(df_r.set_index('rotten_tomatoes_link'), how = 'left')
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df['review_score'] = df['review_score']*5

In [ ]:
df = df.explode('genres')
df.shape

In [ ]:
df.loc[df['genres'] == '', 'genres'] = 'No Genre'

In [ ]:
df_g_sc = df.loc[df['genres'] != 'No Genre'].groupby('genres')['review_score'].median().sort_values(ascending = False)
df_g_sc.index = df_g_sc.index.map(lambda x: x.replace(' ','\n'))
df_g_sc.head()

In [ ]:
fig, ax = plt.subplots(figsize = (25,10))
ax = sns.barplot(x = df_g_sc.index, y = df_g_sc.values)

ax.set_title('Median Review Score for Each Genre', fontsize = 30)
ax.set_xlabel('Genre', fontsize = 20)
ax.set_ylabel('Median Review Score (0 to 5)', fontsize = 20)
ax.tick_params(axis = 'y', labelsize = 15);

In [ ]:
df_genre_edit = copy.deepcopy(df.loc[df['genres'] != 'No Genre'])
df_genre_edit['genres'] = df_genre_edit['genres'].map(lambda x: (' '.join(x.split()[:-1])+'\n'+x.split()[-1]).strip('\n'))

fig2, ax2 = plt.subplots(figsize = (25,10))
ax2 = sns.boxplot(x = 'genres', y = 'review_score', data = df_genre_edit)

ax2.set_title('Review Scores for each Genre', fontsize = 30)
ax2.set_xlabel('Genre', fontsize = 20)
ax2.set_ylabel('Review Scores (0 to 5)', fontsize = 20)
plt.xticks(rotation = 45)
ax2.tick_params(axis = 'both', labelsize = 15);

In [ ]:
df['review_type'].value_counts()

In [ ]:
df['fresh_bool'] = df['review_type'].map({'Fresh':1,'Rotten':0})

In [ ]:
df.loc[:,['review_type','fresh_bool']].head()

In [ ]:
df_g_fb = df.loc[df['genres'] != 'No Genre'].groupby('genres')['fresh_bool'].mean().sort_values(ascending = False)
df_g_fb.index = df_g_fb.index.map(lambda x: x.replace(' ','\n'))
df_g_fb.head()

In [ ]:
fig3, ax3 = plt.subplots(figsize = (25,10))
ax3 = sns.barplot(x = df_g_fb.index, y = df_g_fb.values*100)

ax3.set_title('"Freshness" of Each Genre', fontsize = 30)
ax3.set_xlabel('Genre', fontsize = 20)
ax3.set_ylabel('"Fresh" Percentage', fontsize = 20)
ax3.tick_params(axis = 'y', labelsize = 15);

In [ ]:
df['review_datetime'] = df['review_date'].map(lambda x: time.strptime(x, "%Y-%m-%d"))
df.head(1)

In [ ]:
cutoff = time.strptime('2016-01-01', '%Y-%m-%d')

df_rec = df.loc[df['review_datetime'] >= cutoff]

In [ ]:
print(df['movie_title'].nunique())
print(df_rec['movie_title'].nunique())

In [ ]:
df_rec_g_sc = df_rec.loc[df_rec['genres'] != 'No Genre'].groupby('genres')['review_score'].median().sort_values(ascending = False)
df_rec_g_sc.index = df_g_sc.index.map(lambda x: x.replace(' ','\n'))
df_rec_g_sc.head()

In [ ]:
fig4, ax4 = plt.subplots(figsize = (25,10))
ax4 = sns.barplot(x = df_rec_g_sc.index, y = df_rec_g_sc.values)

ax4.set_title('Recent Median Review Score for Each Genre', fontsize = 30)
ax4.set_xlabel('Genre', fontsize = 20)
ax4.set_ylabel('Median Review Score (0 to 5)', fontsize = 20)
ax4.tick_params(axis = 'y', labelsize = 15);

In [ ]:
df_rec_genre_edit = copy.deepcopy(df_rec.loc[df_rec['genres'] != 'No Genre'])
df_rec_genre_edit['genres'] = df_rec_genre_edit['genres'].map(lambda x: (' '.join(x.split()[:-1])+'\n'+x.split()[-1]).strip('\n'))

fig5, ax5 = plt.subplots(figsize = (25,10))
ax5 = sns.boxplot(x = 'genres', y = 'review_score', data = df_rec_genre_edit)

ax5.set_title('Recent Review Scores for each Genre', fontsize = 30)
ax5.set_xlabel('Genre', fontsize = 20)
ax5.set_ylabel('Review Scores (0 to 5)', fontsize = 20)
plt.xticks(rotation = 45)
ax5.tick_params(axis = 'both', labelsize = 15);

In [ ]:
df_rec_g_fb = df_rec.loc[df_rec['genres'] != 'No Genre'].groupby('genres')['fresh_bool'].mean().sort_values(ascending = False)
df_rec_g_fb.index = df_rec_g_fb.index.map(lambda x: x.replace(' ','\n'))
df_rec_g_fb.head()

In [ ]:
fig6, ax6 = plt.subplots(figsize = (25,10))
ax6 = sns.barplot(x = df_rec_g_fb.index, y = df_rec_g_fb.values*100)

ax6.set_title('Recent "Freshness" of Each Genre', fontsize = 30)
ax6.set_xlabel('Genre', fontsize = 20)
ax6.set_ylabel('"Fresh" Percentage', fontsize = 20)
ax6.tick_params(axis = 'y', labelsize = 15);